## Лекция 2  BM5    

## Функция ранжирования bm25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{TF(q_i,D)*(k+1)}{TF(q_i,D)+k(1-b+b\frac{l(d)}{avgdl})} $$ 
где   
>$TF(q_i,D)$ - частота слова $q_i$ в документе $D$      
$l(d)$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k$ и $b$ — свободные коэффициенты, обычно их выбирают как $k$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ - это модернизированная версия IDF: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [3]:
import collections
import nltk
import re
import csv
import numpy
import time
from math import log
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [4]:
nltk.download("stopwords")
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/victoriaregina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
corpus = []
queries = []
length = []
ids = []
dic = {}
texts = {}
rel = {}
with open('/Users/victoriaregina/Downloads/quora_question_pairs_rus.csv', 'r') as file:
    csv_reader = csv.reader(file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        normal_forms_docs = []
        normal_forms_queries = []
        if line_count == 0:
            line_count += 1
        else:
            if line_count<=1000:
                for t in tokenizer.tokenize(row[2]):
                    t = morph.parse(t)[0]
                    if t.normal_form not in russian_stopwords and t.normal_form != 'это' and not re.match(r'[0-9A-Za-z]', t.normal_form):
                        normal_forms_docs.append(t.normal_form)
                        dic[row[0]] = len(normal_forms_docs)
                        st_doc = ' '.join(normal_forms_docs)
                        texts[row[0]] = st_doc
                corpus.append(st_doc)
                ids.append(row[0])
                length.append(len(normal_forms_docs))
                for t in tokenizer.tokenize(row[1]):
                    t = morph.parse(t)[0]
                    if t.normal_form not in russian_stopwords and t.normal_form != 'это' and not re.match(r'[0-9A-Za-z]', t.normal_form):
                        normal_forms_queries.append(t.normal_form)
                        st_query = ' '.join(normal_forms_queries)
                rel[(st_query, row[0])] = row[3]
                queries.append(st_query)        
            else:
                break
            line_count += 1
    avlen = numpy.mean(length) #средняя длина документа
    N = line_count - 1 #всего документов в коллекции

In [6]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
X = vec.fit_transform(corpus)

df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names(), index=ids)

In [7]:
df['sum'] = df.sum(axis=1) 
tf_table = df.div(df['sum'], axis=0) 
tf_table = tf_table.fillna(0)

In [8]:
#так считаем кол-во документов с этим словом
d = df.isin([0])
nqi = {}
for c in vec.get_feature_names():
    s = N - sum(d[c]) #кол-во документов с этим словом
    nqi[c] = s

### __Задача 1__:    
Напишите два поисковика на *BM25*. Один через подсчет метрики по формуле для каждой пары слово-документ, второй через умножение матрицы на вектор. 

Сравните время работы поиска на 100к запросах. В качестве корпуса возьмем 
[Quora question pairs](https://www.kaggle.com/loopdigga/quora-question-pairs-russian).

# Подсчет метрики по формуле для каждой пары слово-документ

In [11]:
def bm25(i, q, k, b, N, avlen, nqi) -> float:
    try:
        A = log((N - nqi[q] + 0.5)/(nqi[q] + 0.5))
        B = tf_table.at[str(i), q] * (k+1)/(tf_table.at[str(i), q] + k*(1 - b + b*(N/avlen)))
    except KeyError:
        n = 0
        A = log((N - n + 0.5)/(n + 0.5))
        B = 0
    score = A * B
    return score

In [12]:
k = 2.0
b = 0.75
start_time = time.clock()
for query in queries:
    bm25_dic = {}
    for i in ids:
        score = []
        for q in query.split():
            sp = bm25(i, q, k, b, N, avlen, nqi)
            score.append(sp)
    s = numpy.sum(score)
print("Затраченное время для подсчета BM25", "{:g} s".format(time.clock() - start_time))

Затраченное время для подсчета BM25 81.8839 s


# Подсчет метрики через умножение матрицы на вектор

In [13]:
corpus = []
queries = []
length = []
ids = []
dic = {}
texts = {}
rel = {}
with open('/Users/victoriaregina/Downloads/quora_question_pairs_rus.csv', 'r') as file:
    csv_reader = csv.reader(file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        normal_forms_docs = []
        normal_forms_queries = []
        if line_count == 0:
            line_count += 1
        else:
            if line_count<=100:
                for t in tokenizer.tokenize(row[2]):
                    t = morph.parse(t)[0]
                    if t.normal_form not in russian_stopwords and t.normal_form != 'это' and not re.match(r'[0-9A-Za-z]', t.normal_form):
                        normal_forms_docs.append(t.normal_form)
                        dic[row[0]] = len(normal_forms_docs)
                        st_doc = ' '.join(normal_forms_docs)
                        texts[row[0]] = st_doc
                corpus.append(st_doc)
                ids.append(row[0])
                length.append(len(normal_forms_docs))
                for t in tokenizer.tokenize(row[1]):
                    t = morph.parse(t)[0]
                    if t.normal_form not in russian_stopwords and t.normal_form != 'это' and not re.match(r'[0-9A-Za-z]', t.normal_form):
                        normal_forms_queries.append(t.normal_form)
                        st_query = ' '.join(normal_forms_queries)
                rel[(st_query, row[0])] = row[3]
                queries.append(st_query)        
            else:
                break
            line_count += 1
    avlen = numpy.mean(length) #средняя длина документа
    N = line_count - 1 #всего документов в коллекции

In [14]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
X = vec.fit_transform(corpus)

df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names(), index=ids)

In [15]:
def bm25(i, q, k, b, N, avlen, nqi) -> float:
    try:
        A = log((N - nqi[q] + 0.5)/(nqi[q] + 0.5))
        B = tf_table.at[str(i), q] * (k+1)/(tf_table.at[str(i), q] + k*(1 - b + b*(N/avlen)))
    except KeyError:
        n = 0
        A = log((N - n + 0.5)/(n + 0.5))
        B = 0
    score = A * B
    return score

In [16]:
df['sum'] = df.sum(axis=1) 
tf_table = df.div(df['sum'], axis=0) 
tf_table = tf_table.fillna(0)

In [17]:
#так считаем кол-во документов с этим словом
d = df.isin([0])
nqi = {}
for c in vec.get_feature_names():
    s = N - sum(d[c]) #кол-во документов с этим словом
    nqi[c] = s

In [33]:
k = 2.0
b = 0.75
matrix = []
start_time = time.clock()
for query in queries:
    bm25_dic = {}
    for i in ids:
        mat = {}
        score = []
        for q in query.split():
            sp = bm25(i, q, k, b, N, avlen, nqi)
            mat[q] = sp
        matrix.append(mat)

print("Затраченное время для подсчета BM25", "{:g} s".format(time.clock() - start_time))

Затраченное время для подсчета BM25 0.63531 s


In [35]:
df3 = pd.DataFrame(matrix) #создаем матрицу со значением BM25

In [36]:
bm25_mat = df3.fillna(0)

# Задание 2


Выведите 10 первых результатов и их близость по метрике BM25 по запросу **рождественские каникулы** на нашем корпусе  Quora question pairs. 

In [24]:
for t in tokenizer.tokenize('рождественские каникулы'):
    t = morph.parse(t)[0]
    if t.normal_form not in russian_stopwords and t.normal_form != 'это' and not re.match(r'[0-9A-Za-z]', t.normal_form):
        normal_forms_queries.append(t.normal_form)
        st_query = ' '.join(normal_forms_queries)

In [25]:
k = 2.0
b = 0.75
accuracy = 0
bm25_dic = {}
for i in ids:
    score = []
    for q in st_query.split():
        sp = bm25(i, q, k, b, N, avlen, nqi)
        score.append(sp)
    s = numpy.sum(score)
    bm25_dic[s] = (query, i)
m = list(bm25_dic.keys())
m = sorted(m)
for p in m[-10:]: #берем топ 10 документов по метрике bm25
    t = bm25_dic[p]
    print('Вот релевантный документ: '+ t[0] +'; его близость по метрике BM25 равна', p)

Вот релевантный документ: мочь преобразовать необработанный файл фотография; его близость по метрике BM25 равна 0.0


### __Задача 3__:    

Посчитайте точность поиска при 
1. BM25, b=0.75 
2. BM15, b=0 
3. BM11, b=1

In [26]:
def acc(a, k, metrika):
    accuracy = 0
    for query in queries:
        bm25_dic = {}
        for i in ids:
            score = []
            for q in query.split():
                sp = bm25(i, q, k, metrika, N, avlen, nqi)
                score.append(sp)
            s = numpy.sum(score)
            bm25_dic[s] = (query, i)
        m = list(bm25_dic.keys())
        m = sorted(m)
        for p in m[-5:]: #берем топ 5 документов по метрике bm25
            t = bm25_dic[p]
            try:
                if rel[t] == str(1) and p > 0:
                    accuracy += 1
            except KeyError:
                pass
    gen_accuracy = accuracy/N
    a[metrika] = gen_accuracy
    return a

In [27]:
a = {}
k = 2.0
for i in [0.75, 0, 1]:
    acc(a, k, i)

In [37]:
print('Точность поиска при b=0.75 равна', a[0.75])
print('Точность поиска при b=0 равна', a[0])
print('Точность поиска при b=1 равна', a[1])

Точность поиска при b=0.75 равна 0.319
Точность поиска при b=0 равна 0.319
Точность поиска при b=1 равна 0.319
